In [1]:
import agentpy as ap
import random
import IPython
import numpy as np

# For using ggplot from r for better figures
from pyper import *
import os
r = R(RCMD="/Library/Frameworks/R.framework/Resources/bin/R", use_pandas=True)

Implements a basic agent-based version of the Deffuant opinion dynamics model using agentspy. Fully mixed population (non-spatial and non-topological).

In [2]:
class Person(ap.Agent):

    def setup(self):
        """ Initialise new agent variables at agent creation """

        self.opinion = random.uniform(-1,1)
        self.uncertainty = np.random.beta(self.p.alpha, self.p.beta)
        self.new_opinion = self.opinion # Used for synchronous updating

    def opinion_influence(self):
        """ This agent influences other agents in the population """

        # Randomly select a fixed number of agents as partners to influence
        partners = self.model.pop.random(self.p.num_partners, replace=False)
        for partner in partners:
            opinion_diff = abs(self.opinion - partner.opinion)
            if opinion_diff <= self.uncertainty:
                opinion_change = self.p.mu * (self.opinion - partner.opinion)
                partner.new_opinion = partner.opinion + opinion_change

    def update_opinion(self):
        self.opinion = self.new_opinion

class OpinionModel(ap.Model):

    def setup(self):
        """ Initialise the model and setup the agent population """
        
        # Create agent population
        self.pop = ap.AgentList(self, self.p.pop_size, Person)

    def update(self):
        """ Called after setup and every step """

        # Synchronous updating of opinions
        self.pop.update_opinion()
        # Record stats
        self.pop.record("opinion")

    def step(self):
        """ Model events per timestep """

        self.pop.opinion_influence()

    def end(self):
        """ Termination conditions """
        return

In [3]:
parameters = {
    'pop_size': 200,
    'steps': 200,
    'num_partners': 4,
    'mu': 0.05,
    'alpha': 3,
    'beta': 6
}

In [4]:
model = OpinionModel(parameters)
results = model.run()

Completed: 200 steps
Run time: 0:00:00.458445
Simulation finished


In [5]:
# Get all logged data for Person agents
results = results.variables.Person
# Convert the resulting Pandas series to a dataFrame
results = results.opinion.to_frame().reset_index()

In [6]:
# Use R for better plotting
r.assign("results_df",results)
r.assign("pop_size",parameters["pop_size"])
r("""
library(tidyverse)

# Make similar starting values have similar colours
colours <- results_df %>% 
    group_by(obj_id) %>% 
    filter(t == 1) %>%
    arrange(opinion) %>%
    add_column(colour = 1:pop_size) %>%
    arrange(obj_id) %>%
    select(obj_id, colour)

results_df <- left_join(results_df, colours, by="obj_id")

ggplot(results_df,aes(x=t,y=opinion,color=factor(colour))) +
      geom_line(size=0.6,alpha=0.6) +
      theme_minimal() +
      theme(legend.position="none") +
      theme(axis.line = element_line(color="black")) +
      xlab("Timestep") + ylab("Opinion")
ggsave("opinion_dynamic_run.png")
""")

'try({\n+ library(tidyverse)\n+ \n+ # Make similar starting values have similar colours\n+ colours <- results_df %>% \n+     group_by(obj_id) %>% \n+     filter(t == 1) %>%\n+     arrange(opinion) %>%\n+     add_column(colour = 1:pop_size) %>%\n+     arrange(obj_id) %>%\n+     select(obj_id, colour)\n+ \n+ results_df <- left_join(results_df, colours, by="obj_id")\n+ \n+ ggplot(results_df,aes(x=t,y=opinion,color=factor(colour))) +\n+       geom_line(size=0.6,alpha=0.6) +\n+       theme_minimal() +\n+       theme(legend.position="none") +\n+       theme(axis.line = element_line(color="black")) +\n+       xlab("Timestep") + ylab("Opinion")\n+ ggsave("opinion_dynamic_run.png")\n+ })\n── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──\n✔ ggplot2 3.3.5     ✔ purrr   0.3.4\n✔ tibble  3.1.6     ✔ dplyr   1.0.7\n✔ tidyr   1.1.4     ✔ stringr 1.4.0\n✔ readr   2.1.1     ✔ forcats 0.5.1\n── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──\

In [7]:
os.system("open opinion_dynamic_run.png")

0